In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last" #"last expr -> all로 바꾸면 전체가 나온다. "
pd.options.display.max_columns = 200
pd.options.display.max_rows = 250
pd.options.display.max_colwidth = 100

In [2]:
from sklearn.preprocessing import MinMaxScaler

from six.moves import xrange
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU
from keras.layers import Lambda
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import RMSprop,adam
from keras.callbacks import History
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user/anaconda3/envs/park/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user/anaconda3/envs/park/l

In [3]:
outcome = pd.read_csv("./backup1007/scaled_total_input191204.csv")

outcome=outcome.drop(["Unnamed: 0"], axis=1)

/home/user/anaconda3/envs/park/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (238,244) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
outcome.head()

,idx,admday,labday,time(round),SBP(std),DBP(std),PR(std),RR(std),BT(std),pCO2(std),pO2(std),Base Excess(std),Bicarbonate(std),O2 Saturation(std),pH ABGA(std),유산(동맥혈)(std),SBP(count),DBP(count),PR(count),RR(count),BT(count),pCO2(count),pO2(count),Base Excess(count),Bicarbonate(count),O2 Saturation(count),pH ABGA(count),유산(동맥혈)(count),SBP(diff),DBP(diff),PR(diff),RR(diff),BT(diff),Hb(diff),Platelet(diff),WBC(diff),RBC(diff),Hct(diff),MCH(diff),MCHC(diff),MCV(diff),RDW(diff),PDW(diff),MPV(diff),E-ANC(diff),E-Eosinophil(diff),E-Basophil(diff),E-Lymphocyte(diff),E-Monocyte(diff),E-Neutrophil(diff),AST (SGOT)(diff),ALT (SGPT)(diff),"Bilirubin,total(diff)",Potassium(diff),Sodium(diff),Chloride(diff),Albumin(diff),Creatinine(diff),Alkaline Phosphatase(diff),Protein(diff),BUN(diff),Glucose(diff),Cholesterol(diff),Total CO2(diff),CRP(quant)(diff),Calcium(diff),Uric acid(diff),IG %(diff),Amylase(diff),Phosphorus(diff),pCO2(diff),pO2(diff),Base Excess(diff),Bicarbonate(diff),O2 Saturation(diff),pH ABGA(diff),eGFR(CKD-EPI)(diff),activated PTT(diff),유산(동맥혈)(diff),Magnesium(diff),SBP(mean),DBP(mean),PR(mean),RR(mean),BT(mean),Hb(mean),Platelet(mean),WBC(mean),RBC(mean),Hct(mean),MCH(mean),MCHC(mean),MCV(mean),RDW(mean),PDW(mean),MPV(mean),E-ANC(mean),E-Eosinophil(mean),E-Basophil(mean),E-Lymphocyte(mean),...,MCV(max),RDW(max),PDW(max),MPV(max),E-ANC(max),E-Eosinophil(max),E-Basophil(max),E-Lymphocyte(max),E-Monocyte(max),E-Neutrophil(max),AST (SGOT)(max),ALT (SGPT)(max),"Bilirubin,total(max)",Potassium(max),Sodium(max),Chloride(max),Albumin(max),Creatinine(max),Alkaline Phosphatase(max),Protein(max),BUN(max),Glucose(max),Cholesterol(max),Total CO2(max),CRP(quant)(max),Calcium(max),Uric acid(max),IG %(max),Amylase(max),Phosphorus(max),pCO2(max),pO2(max),Base Excess(max),Bicarbonate(max),O2 Saturation(max),pH ABGA(max),eGFR(CKD-EPI)(max),activated PTT(max),유산(동맥혈)(max),Magnesium(max),SBP(min),DBP(min),PR(min),RR(min),BT(min),Hb(min),Platelet(min),WBC(min),RBC(min),Hct(min),MCH(min),MCHC(min),MCV(min),RDW(min),PDW(min),MPV(min),E-ANC(min),E-Eosinophil(min),E-Basophil(min),E-Lymphocyte(min),E-Monocyte(min),E-Neutrophil(min),AST (SGOT)(min),ALT (SGPT)(min),"Bilirubin,total(min)",Potassium(min),Sodium(min),Chloride(min),Albumin(min),Creatinine(min),Alkaline Phosphatase(min),Protein(min),BUN(min),Glucose(min),Cholesterol(min),Total CO2(min),CRP(quant)(min),Calcium(min),Uric acid(min),IG %(min),Amylase(min),Phosphorus(min),pCO2(min),pO2(min),Base Excess(min),Bicarbonate(min),O2 Saturation(min),pH ABGA(min),eGFR(CKD-EPI)(min),activated PTT(min),유산(동맥혈)(min),Magnesium(min),exam_cul,code_name2,result,exam_cul3,result3,G,time,censored
0,1166795967,20160815,20160815,1600,0.520866,0.152576,0.190476,0.422577,0.295599,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.142857,0.142857,0.142857,0.142857,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,NaN,-0.021898,-0.011494,-0.08,-0.047619,-0.025641,0.090909,-0.159817,-0.071429,0.084507,0.092308,0.037037,2.220446e-15,0.027027,-0.047619,0.043478,0.047619,-0.102017,-0.066667,-0.066667,0.257778,-0.037037,-0.204225,0.05625,0.029412,0.1875,-0.047619,0.076923,0.142857,0.111111,0.0,-0.019355,0.172414,0.0,0.045802,0.215686,-2.220446e-16,-0.121003,0.259259,0.111111,-0.090909,NaN,0.105263,0.03066,0.140465,0.232493,0.18125,0.186441,0.211073,-0.047619,-0.146067,NaN,NaN,-0.542857,-0.557895,-1.0,0.086957,-0.327586,0.703704,-0.785714,-0.567542,0.877863,0.77193,-0.045872,0.090909,-0.20155,0.136069,0.347826,0.318182,-0.286871,0.265528,0.203585,0.351814,...,-0.20155,0.136069,0.333333,0.318182,-0.288157,0.265528,0.203585,0.351814,-0.826667,-0.022059,-0.465827,-0.480878,-0.584389,0.304348,0.529412,0.052632,1.0,0.285113,-0.318048,0.5,0.380639,-0.204314,-0.295775,0.225352,-1.0,0.714286,-0.304348,-0.604435,NaN,0.095238,-0.006579,-0.772999,0.066667,0.074074,0.128164,-0.1341,0.105785,0.047234,NaN,NaN,-0.714286,-0.551020,-1.0,0.333333,-0.411765,0.703704,-0.785714,-0.565687,0.878327,0.772926,-0.045872,0.090909,-0.20155,0.136069,0.347826,0.318182,-0.286356,0.265528,0.2035

In [5]:
outcome.iloc[:,4:-8]=outcome.iloc[:,4:-8].fillna(0)

In [6]:
outcome.head(20)

,idx,admday,labday,time(round),SBP(std),DBP(std),PR(std),RR(std),BT(std),pCO2(std),pO2(std),Base Excess(std),Bicarbonate(std),O2 Saturation(std),pH ABGA(std),유산(동맥혈)(std),SBP(count),DBP(count),PR(count),RR(count),BT(count),pCO2(count),pO2(count),Base Excess(count),Bicarbonate(count),O2 Saturation(count),pH ABGA(count),유산(동맥혈)(count),SBP(diff),DBP(diff),PR(diff),RR(diff),BT(diff),Hb(diff),Platelet(diff),WBC(diff),RBC(diff),Hct(diff),MCH(diff),MCHC(diff),MCV(diff),RDW(diff),PDW(diff),MPV(diff),E-ANC(diff),E-Eosinophil(diff),E-Basophil(diff),E-Lymphocyte(diff),E-Monocyte(diff),E-Neutrophil(diff),AST (SGOT)(diff),ALT (SGPT)(diff),"Bilirubin,total(diff)",Potassium(diff),Sodium(diff),Chloride(diff),Albumin(diff),Creatinine(diff),Alkaline Phosphatase(diff),Protein(diff),BUN(diff),Glucose(diff),Cholesterol(diff),Total CO2(diff),CRP(quant)(diff),Calcium(diff),Uric acid(diff),IG %(diff),Amylase(diff),Phosphorus(diff),pCO2(diff),pO2(diff),Base Excess(diff),Bicarbonate(diff),O2 Saturation(diff),pH ABGA(diff),eGFR(CKD-EPI)(diff),activated PTT(diff),유산(동맥혈)(diff),Magnesium(diff),SBP(mean),DBP(mean),PR(mean),RR(mean),BT(mean),Hb(mean),Platelet(mean),WBC(mean),RBC(mean),Hct(mean),MCH(mean),MCHC(mean),MCV(mean),RDW(mean),PDW(mean),MPV(mean),E-ANC(mean),E-Eosinophil(mean),E-Basophil(mean),E-Lymphocyte(mean),...,MCV(max),RDW(max),PDW(max),MPV(max),E-ANC(max),E-Eosinophil(max),E-Basophil(max),E-Lymphocyte(max),E-Monocyte(max),E-Neutrophil(max),AST (SGOT)(max),ALT (SGPT)(max),"Bilirubin,total(max)",Potassium(max),Sodium(max),Chloride(max),Albumin(max),Creatinine(max),Alkaline Phosphatase(max),Protein(max),BUN(max),Glucose(max),Cholesterol(max),Total CO2(max),CRP(quant)(max),Calcium(max),Uric acid(max),IG %(max),Amylase(max),Phosphorus(max),pCO2(max),pO2(max),Base Excess(max),Bicarbonate(max),O2 Saturation(max),pH ABGA(max),eGFR(CKD-EPI)(max),activated PTT(max),유산(동맥혈)(max),Magnesium(max),SBP(min),DBP(min),PR(min),RR(min),BT(min),Hb(min),Platelet(min),WBC(min),RBC(min),Hct(min),MCH(min),MCHC(min),MCV(min),RDW(min),PDW(min),MPV(min),E-ANC(min),E-Eosinophil(min),E-Basophil(min),E-Lymphocyte(min),E-Monocyte(min),E-Neutrophil(min),AST (SGOT)(min),ALT (SGPT)(min),"Bilirubin,total(min)",Potassium(min),Sodium(min),Chloride(min),Albumin(min),Creatinine(min),Alkaline Phosphatase(min),Protein(min),BUN(min),Glucose(min),Cholesterol(min),Total CO2(min),CRP(quant)(min),Calcium(min),Uric acid(min),IG %(min),Amylase(min),Phosphorus(min),pCO2(min),pO2(min),Base Excess(min),Bicarbonate(min),O2 Saturation(min),pH ABGA(min),eGFR(CKD-EPI)(min),activated PTT(min),유산(동맥혈)(min),Magnesium(min),exam_cul,code_name2,result,exam_cul3,result3,G,time,censored
0,1166795967,20160815,20160815,1600,0.520866,0.152576,0.190476,0.422577,0.295599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.142857,0.142857,0.142857,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.021898,-0.011494,-0.08,-0.047619,-0.025641,0.090909,-0.159817,-0.071429,0.084507,0.092308,0.037037,2.220446e-15,0.027027,-0.047619,0.043478,0.047619,-0.102017,-0.066667,-0.066667,0.257778,-0.037037,-0.204225,0.05625,0.029412,0.1875,-0.047619,0.076923,0.142857,0.111111,0.000000,-0.019355,0.172414,0.000000,0.045802,0.215686,-2.220446e-16,-0.121003,0.259259,0.111111,-0.090909,0.0,0.105263,0.03066,0.140465,0.232493,0.18125,0.186441,0.211073,-0.047619,-0.146067,0.0,0.0,-0.542857,-0.557895,-1.000000,0.086957,-0.327586,0.703704,-0.785714,-0.567542,0.877863,0.771930,-0.045872,0.090909,-0.201550,0.136069,0.347826,0.318182,-0.286871,0.265528,0.203585,0.351814,...,-0.201550,0.136069,0.333333,0.318182,-0.288157,0.265528,0.203585,0.351814,-0.826667,-0.022059,-0.465827,-0.480878,-0.584389,0.304348,0.529412,0.052632,1.000000,0.285113,-0.318048,0.500000,0.380639,-0.204314,-0.295775,2.253521e-01,-1.000000,0.714286,-0.304348,-0.604435,0.0,0.095238,-0.006579,-0.772999,0.066667,0.074074,0.128164,-0.134100,0.105785,0.047234,0.0,0.0,-7.142857e-01,-0.551020,-1.000000,0.333333,-0.411765,0.703704,-0.785714,-0.565687,0.878327,0.772926,-0.045872,0.090909,-0.201550,0.1

### WBC column

In [7]:
list1=outcome.columns.tolist()

In [8]:
vital_column=outcome.columns[outcome.columns.str.contains('SBP|DBP|BT|PR|RR')]
hct_columns =outcome.columns[outcome.columns.str.contains('Hct|RBC|Hb|MCHC|MCV|MCH|RDW|MPV|PDW|Hemoglobin Index')]
liver_column=outcome.columns[outcome.columns.str.contains('ALT|AST|Bilirubin,total|Alkaline Phosphatase|Cholesterol|activated PTT|Icteric Index|Lipemia Index|Albumin|Protein')]
electro_column=outcome.columns[outcome.columns.str.contains('Calcium|Phosphorus|Potassium|Sodium|Chloride|Magnesium')]
creatinine_column=outcome.columns[outcome.columns.str.contains('eGFR|Creatinine|BUN|Uric acid')]
abga_column=outcome.columns[outcome.columns.str.contains('Glucose|Total CO2|Saturation|pCO2|pO2|Bicarbonate|Base Excess|pH ABGA|유산')]
crp_column=outcome.columns[outcome.columns.str.contains('CRP|Amylase')]
wbc_column=outcome.columns[outcome.columns.str.contains('WBC|E-Neutrophil|E-ANC|E-Eosinophil|E-Monocyte|E-Lymphocyte|E-Basophil|IG')]

In [9]:
list1=[x for x in list1 if x not in vital_column]
list1=[x for x in list1 if x not in hct_columns]
list1=[x for x in list1 if x not in liver_column]
list1=[x for x in list1 if x not in electro_column]
list1=[x for x in list1 if x not in creatinine_column]
list1=[x for x in list1 if x not in abga_column]
list1=[x for x in list1 if x not in crp_column]
list1=[x for x in list1 if x not in wbc_column]
list1

['idx',
 'admday',
 'labday',
 'time(round)',
 'Platelet(diff)',
 'Platelet(mean)',
 'Platelet(max)',
 'Platelet(min)',
 'exam_cul',
 'code_name2',
 'result',
 'exam_cul3',
 'result3',
 'G',
 'time',
 'censored']

In [10]:
from keras.models import model_from_json 
json_file = open("./scale_emb_folder/rnn_vital_encoding_12_2dense.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)


loaded_model.load_weights("./scale_emb_folder/rnn_vital_encoding_12_2dense.h5") 
print("Loaded model from disk")


loaded_model.summary()

input_img = Input(shape=(30,))

encoded1=loaded_model.layers[1]
encoded2=loaded_model.layers[2]

encoder= Model(input_img, encoded2(encoded1(input_img)))

vital=outcome[vital_column]

encoded_vital=encoder.predict(vital)






Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_61 (InputLayer)        (None, 30)                0         
_________________________________________________________________
encoder1 (Dense)             (None, 64)                1984      
_________________________________________________________________
encoder2 (Dense)             (None, 12)                780       
_________________________________________________________________
decoder1 (Dense)             (None, 64)                832       
_________________________________________________________________
decoder2 (Dense)             (None, 30)                1950      
Total params: 5,546
Trainable params: 5,546
Non-trainable params: 0
_________________________________________________________________


In [11]:
from keras.models import model_from_json 
json_file = open("./scale_emb_folder/rnn_wbc_encoding_12_2dense.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)


loaded_model.load_weights("./scale_emb_folder/rnn_wbc_encoding_12_2dense.h5") 
print("Loaded model from disk")


loaded_model.summary()

input_img = Input(shape=(32,))

encoded1=loaded_model.layers[1]
encoded2=loaded_model.layers[2]

encoder= Model(input_img, encoded2(encoded1(input_img)))

wbc=outcome[wbc_column]

encoded_wbc=encoder.predict(wbc)

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        (None, 32)                0         
_________________________________________________________________
encoder1 (Dense)             (None, 64)                2112      
_________________________________________________________________
encoder2 (Dense)             (None, 12)                780       
_________________________________________________________________
decoder1 (Dense)             (None, 64)                832       
_________________________________________________________________
decoder2 (Dense)             (None, 32)                2080      
Total params: 5,804
Trainable params: 5,804
Non-trainable params: 0
_________________________________________________________________


In [12]:
from keras.models import model_from_json 
json_file = open("./scale_emb_folder/rnn_hct_encoding_12_2dense.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("./scale_emb_folder/rnn_hct_encoding_12_2dense.h5") 
print("Loaded model from disk")


loaded_model.summary()

input_img = Input(shape=(36,))

encoded1=loaded_model.layers[1]
encoded2=loaded_model.layers[2]

encoder= Model(input_img, encoded2(encoded1(input_img)))

hct=outcome[hct_columns]

encoded_hct=encoder.predict(hct)

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_53 (InputLayer)        (None, 36)                0         
_________________________________________________________________
encoder1 (Dense)             (None, 64)                2368      
_________________________________________________________________
encoder2 (Dense)             (None, 12)                780       
_________________________________________________________________
decoder1 (Dense)             (None, 64)                832       
_________________________________________________________________
decoder2 (Dense)             (None, 36)                2340      
Total params: 6,320
Trainable params: 6,320
Non-trainable params: 0
_________________________________________________________________


In [13]:
from keras.models import model_from_json 
json_file = open("./scale_emb_folder/rnn_electro_encoding_12_2dense.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)


loaded_model.load_weights("./scale_emb_folder/rnn_electro_encoding_12_2dense.h5") 
print("Loaded model from disk")


loaded_model.summary()

input_img = Input(shape=(24,))

encoded1=loaded_model.layers[1]
encoded2=loaded_model.layers[2]

encoder= Model(input_img, encoded2(encoded1(input_img)))

electro=outcome[electro_column]
encoded_electro=encoder.predict(electro)

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 24)                0         
_________________________________________________________________
encoder1 (Dense)             (None, 64)                1600      
_________________________________________________________________
encoder2 (Dense)             (None, 12)                780       
_________________________________________________________________
decoder1 (Dense)             (None, 64)                832       
_________________________________________________________________
decoder2 (Dense)             (None, 24)                1560      
Total params: 4,772
Trainable params: 4,772
Non-trainable params: 0
_________________________________________________________________


In [14]:
json_file = open("./scale_emb_folder/rnn_creatinine_encoding_12_2dense.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)


loaded_model.load_weights("./scale_emb_folder/rnn_creatinine_encoding_12_2dense.h5") 
print("Loaded model from disk")


loaded_model.summary()

input_img = Input(shape=(16,))

encoded1=loaded_model.layers[1]
encoded2=loaded_model.layers[2]

encoder= Model(input_img, encoded2(encoded1(input_img)))

creatinine=outcome[creatinine_column]
encoded_creatinine=encoder.predict(creatinine)

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 16)                0         
_________________________________________________________________
encoder1 (Dense)             (None, 64)                1088      
_________________________________________________________________
encoder2 (Dense)             (None, 12)                780       
_________________________________________________________________
decoder1 (Dense)             (None, 64)                832       
_________________________________________________________________
decoder2 (Dense)             (None, 16)                1040      
Total params: 3,740
Trainable params: 3,740
Non-trainable params: 0
_________________________________________________________________


In [15]:
json_file = open("./scale_emb_folder/rnn_crp_encoding_12_2dense.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)


loaded_model.load_weights("./scale_emb_folder/rnn_crp_encoding_12_2dense.h5") 
print("Loaded model from disk")


loaded_model.summary()

input_img = Input(shape=(8,))

encoded1=loaded_model.layers[1]
encoded2=loaded_model.layers[2]

encoder= Model(input_img, encoded2(encoded1(input_img)))

crp=outcome[crp_column]
encoded_crp=encoder.predict(crp)

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        (None, 8)                 0         
_________________________________________________________________
encoder1 (Dense)             (None, 64)                576       
_________________________________________________________________
encoder2 (Dense)             (None, 12)                780       
_________________________________________________________________
decoder1 (Dense)             (None, 64)                832       
_________________________________________________________________
decoder2 (Dense)             (None, 8)                 520       
Total params: 2,708
Trainable params: 2,708
Non-trainable params: 0
_________________________________________________________________


In [16]:
json_file = open("./scale_emb_folder/rnn_abga_encoding_12_2dense.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)


loaded_model.load_weights("./scale_emb_folder/rnn_abga_encoding_12_2dense.h5") 
print("Loaded model from disk")


loaded_model.summary()

input_img = Input(shape=(50,))

encoded1=loaded_model.layers[1]
encoded2=loaded_model.layers[2]

encoder= Model(input_img, encoded2(encoded1(input_img)))

abga=outcome[abga_column]
encoded_abga=encoder.predict(abga)

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 50)                0         
_________________________________________________________________
encoder1 (Dense)             (None, 64)                3264      
_________________________________________________________________
encoder2 (Dense)             (None, 12)                780       
_________________________________________________________________
decoder1 (Dense)             (None, 64)                832       
_________________________________________________________________
decoder2 (Dense)             (None, 50)                3250      
Total params: 8,126
Trainable params: 8,126
Non-trainable params: 0
_________________________________________________________________


In [17]:
json_file = open("./scale_emb_folder/rnn_liver_encoding_12_2dense.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
loaded_model = model_from_json(loaded_model_json)


loaded_model.load_weights("./scale_emb_folder/rnn_liver_encoding_12_2dense.h5") 
print("Loaded model from disk")


loaded_model.summary()

input_img = Input(shape=(32,))

encoded1=loaded_model.layers[1]
encoded2=loaded_model.layers[2]

encoder= Model(input_img, encoded2(encoded1(input_img)))

liver=outcome[liver_column]
encoded_liver=encoder.predict(liver)

Loaded model from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 32)                0         
_________________________________________________________________
encoder1 (Dense)             (None, 64)                2112      
_________________________________________________________________
encoder2 (Dense)             (None, 12)                780       
_________________________________________________________________
decoder1 (Dense)             (None, 64)                832       
_________________________________________________________________
decoder2 (Dense)             (None, 32)                2080      
Total params: 5,804
Trainable params: 5,804
Non-trainable params: 0
_________________________________________________________________


In [18]:

vital1 = pd.DataFrame(encoded_vital)
index1 = pd.MultiIndex.from_product([['vital'],vital1.columns])
vital2 = pd.DataFrame(encoded_vital, columns = index1)

hct1 = pd.DataFrame(encoded_hct)
hct_index = pd.MultiIndex.from_product([['hct'],hct1.columns])
hct2 = pd.DataFrame(encoded_hct, columns = hct_index)

elec_ = pd.DataFrame(encoded_electro)
elec_12 = pd.MultiIndex.from_product([['electro'],elec_.columns])
elec_2 = pd.DataFrame(encoded_electro, columns = elec_12)

cr1 = pd.DataFrame(encoded_creatinine)
cr_idx = pd.MultiIndex.from_product([['creatinine'],cr1.columns])
cr2 = pd.DataFrame(encoded_creatinine, columns = cr_idx)

crp1 = pd.DataFrame(encoded_crp)
crp_idx = pd.MultiIndex.from_product([['crp'],crp1.columns])
crp2 = pd.DataFrame(encoded_crp, columns = crp_idx)

wbc1 = pd.DataFrame(encoded_wbc)
wbc_dx = pd.MultiIndex.from_product([['wbc'],wbc1.columns])
wbc2 = pd.DataFrame(encoded_wbc, columns = wbc_dx)

abga1 = pd.DataFrame(encoded_abga)
abga_idx = pd.MultiIndex.from_product([['abga'],abga1.columns])
abga2 = pd.DataFrame(encoded_abga, columns = abga_idx)

liver1 = pd.DataFrame(encoded_liver)
liver_idx = pd.MultiIndex.from_product([['liver'],liver1.columns])
liver2 = pd.DataFrame(encoded_liver, columns = liver_idx)

all_cat = pd.concat([vital2, hct2,elec_2,cr2, crp2, wbc2, abga2, liver2], axis=1)

In [19]:
all_cat.head()

vital                                                              \
         0         1         2         3         4         5         6    
0  0.877540  0.848782  1.489119  1.756759  1.130512  2.058580  1.145750   
1  0.643219  0.880033  0.962514  1.340707  0.574145  1.134869  1.041484   
2  0.633339  0.879566  0.597355  1.417726  0.566608  1.127686  1.283000   
3  1.222373  0.903579  1.965943  1.916501  0.988427  1.841545  1.565406   
4  0.570073  1.242540  1.116480  1.330996  0.670641  1.310422  0.719681   

                                                          hct            \
         7         8         9         10        11        0         1    
0  1.006884  1.500511  1.752685  0.647886  1.188586  1.149542  1.474626   
1  0.998520  0.971610  1.710735  0.360490  1.525325  1.149542  1.474626   
2  0.796019  0.995784  1.413689  0.218041  1.083368  1.149542  1.474626   
3  1.076945  1.743122  1.935657  0.385933  1.148217  1.149542  1.474626   
4  1.503995  1.271435  2.055010  0.046490  1.207233  1.149542  1.474626   

                                                                              \
         2         3         4         5    6         7         8         9    
0  0.972831  1.088646  1.663667  1.614732  0.0  1.345414  1.258846  1.187768   
1  0.972831  1.088646  1.663667  1.614732  0.0  1.345414  1.258846  1.187768   
2  0.972831  1.088646  1.663667  1.614732  0.0  1.345414  1.258846  1.187768   
3  0.972831  1.088646  1.663667  1.614732  0.0  1.345414  1.258846  1.187768   
4  0.972831  1.088646  1.663667  1.614732  0.0  1.345414  1.258846  1.187768   

                        electro                                         \
         10        11        0        1         2         3         4    
0  2.135049  1.535705  1.651898  1.37116  1.694416  1.833908  1.383186   
1  2.135049  1.535705  1.651898  1.37116  1.694416  1.833908  1.383186   
2  2.135049  1.535705  1.651898  1.37116  1.694416  1.833908  1.383186   
3  2.135049  1.535705  1.651898  1.37116  1.694416  1.833908  1.383186   
4  2.135049  1.535705  1.651898  1.37116  1.694416  1.833908  1.383186   

                                                                        \
         5         6        7         8         9         10        11   
0  0.975757  0.987423  1.48969  1.549485  1.210805  1.061204  1.279428   
1  0.975757  0.987423  1.48969  1.549485  1.210805  1.061204  1.279428   
2  0.975757  0.987423  1.48969  1.549485  1.210805  1.061204  1.279428   
3  0.975757  0.987423  1.48969  1.549485  1.210805  1.061204  1.279428   
4  0.975757  0.987423  1.48969  1.549485  1.210805  1.061204  1.279428   

  creatinine                                                                   \
          0         1         2         3         4    5         6         7    
0   1.540031  1.131879  1.213531  1.571267  0.955697  0.0  0.236189  1.564333   
1   1.540031  1.131879  1.213531  1.571267  0.955697  0.0  0.236189  1.564333   
2   1.540031  1.131879  1.213531  1.571267  0.955697  0.0  0.236189  1.564333   
3   1.540031  1.131879  1.213531  1.571267  0.955697  0.0  0.236189  1.564333   
4   1.540031  1.131879  1.213531  1.571267  0.955697  0.0  0.236189  1.564333   

                                      crp                                \
         8         9         10   11   0    1         2         3    4    
0  1.229216  1.374299  0.444511  0.0  0.0  0.0  0.990211  0.895441  0.0   
1  1.229216  1.374299  0.444511  0.0  0.0  0.0  0.990211  0.895441  0.0   
2  1.229216  1.374299  0.444511  0.0  0.0  0.0  0.990211  0.895441  0.0   
3  1.229216  1.374299  0.444511  0.0  0.0  0.0  0.990211  0.895441  0.0   
4  1.229216  1.374299  0.444511  0.0  0.0  0.0  0.990211  0.895441  0.0   

                                                                    wbc  \
         5    6         7         8         9         10   11        0    
0  2.033415  0.0  0.154487  0.387764  1.537237  0.626813  0.0  1.147379   
1  2.033415  0.0  0.154487  0.387764

In [21]:
plateletcolumn=outcome.columns[outcome.columns.str.contains('Platelet')]

In [22]:
platelet = outcome[plateletcolumn]
plt_idx = pd.MultiIndex.from_product([['plt'],platelet.columns])
plt2 = pd.DataFrame(platelet.get_values(), columns = plt_idx)


/home/user/anaconda3/envs/park/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.values' or 'np.asarray(..)' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
name=outcome[['idx','admday','labday','time(round)']]

In [24]:
label=outcome.iloc[:,-8:]

In [25]:
all_cat = pd.concat([name, plt2,all_cat, label],axis=1)

In [26]:
all_cat.head()

,idx,admday,labday,time(round),"(plt, Platelet(diff))","(plt, Platelet(mean))","(plt, Platelet(max))","(plt, Platelet(min))","(vital, 0)","(vital, 1)","(vital, 2)","(vital, 3)","(vital, 4)","(vital, 5)","(vital, 6)","(vital, 7)","(vital, 8)","(vital, 9)","(vital, 10)","(vital, 11)","(hct, 0)","(hct, 1)","(hct, 2)","(hct, 3)","(hct, 4)","(hct, 5)","(hct, 6)","(hct, 7)","(hct, 8)","(hct, 9)","(hct, 10)","(hct, 11)","(electro, 0)","(electro, 1)","(electro, 2)","(electro, 3)","(electro, 4)","(electro, 5)","(electro, 6)","(electro, 7)","(electro, 8)","(electro, 9)","(electro, 10)","(electro, 11)","(creatinine, 0)","(creatinine, 1)","(creatinine, 2)","(creatinine, 3)","(creatinine, 4)","(creatinine, 5)","(creatinine, 6)","(creatinine, 7)","(creatinine, 8)","(creatinine, 9)","(creatinine, 10)","(creatinine, 11)","(crp, 0)","(crp, 1)","(crp, 2)","(crp, 3)","(crp, 4)","(crp, 5)","(crp, 6)","(crp, 7)","(crp, 8)","(crp, 9)","(crp, 10)","(crp, 11)","(wbc, 0)","(wbc, 1)","(wbc, 2)","(wbc, 3)","(wbc, 4)","(wbc, 5)","(wbc, 6)","(wbc, 7)","(wbc, 8)","(wbc, 9)","(wbc, 10)","(wbc, 11)","(abga, 0)","(abga, 1)","(abga, 2)","(abga, 3)","(abga, 4)","(abga, 5)","(abga, 6)","(abga, 7)","(abga, 8)","(abga, 9)","(abga, 10)","(abga, 11)","(liver, 0)","(liver, 1)","(liver, 2)","(liver, 3)","(liver, 4)","(liver, 5)","(liver, 6)","(liver, 7)","(liver, 8)","(liver, 9)","(liver, 10)","(liver, 11)",exam_cul,code_name2,result,exam_cul3,result3,G,time,censored
0,1166795967,20160815,20160815,1600,-0.159817,-0.785714,-0.789077,-0.785714,0.877540,0.848782,1.489119,1.756759,1.130512,2.058580,1.145750,1.006884,1.500511,1.752685,0.647886,1.188586,1.149542,1.474626,0.972831,1.088646,1.663667,1.614732,0.0,1.345414,1.258846,1.187768,2.135049,1.535705,1.651898,1.37116,1.694416,1.833908,1.383186,0.975757,0.987423,1.48969,1.549485,1.210805,1.061204,1.279428,1.540031,1.131879,1.213531,1.571267,0.955697,0.0,0.236189,1.564333,1.229216,1.374299,0.444511,0.0,0.0,0.0,0.990211,0.895441,0.0,2.033415,0.0,0.154487,0.387764,1.537237,0.626813,0.0,1.147379,1.564688,1.128007,1.76054,1.1189,1.230449,1.441725,2.127261,1.031093,1.284279,1.891631,1.166048,1.596105,1.59585,0.866867,1.638885,1.3785,1.29153,1.003529,1.171839,1.798226,1.681464,1.323524,1.614375,2.386357,1.334399,1.178799,0.378336,1.091599,1.903982,1.161755,1.524952,1.029865,0.0,2.193275,1.638233,0,NaN,0,0,0,False,15,0
1,1166795967,20160815,20160815,2300,-0.159817,-0.785714,-0.789077,-0.785714,0.643219,0.880033,0.962514,1.340707,0.574145,1.134869,1.041484,0.998520,0.971610,1.710735,0.360490,1.525325,1.149542,1.474626,0.972831,1.088646,1.663667,1.614732,0.0,1.345414,1.258846,1.187768,2.135049,1.535705,1.651898,1.37116,1.694416,1.833908,1.383186,0.975757,0.987423,1.48969,1.549485,1.210805,1.061204,1.279428,1.540031,1.131879,1.213531,1.571267,0.955697,0.0,0.236189,1.564333,1.229216,1.374299,0.444511,0.0,0.0,0.0,0.990211,0.895441,0.0,2.033415,0.0,0.154487,0.387764,1.537237,0.626813,0.0,1.147379,1.564688,1.128007,1.76054,1.1189,1.230449,1.441725,2.127261,1.031093,1.284279,1.891631,1.166048,1.596105,1.59585,0.866867,1.638885,1.3785,1.29153,1.003529,1.171839,1.798226,1.681464,1.323524,1.614375,2.386357,1.334399,1.178799,0.378336,1.091599,1.903982,1.161755,1.524952,1.029865,0.0,2.193275,1.638233,0,NaN,0,0,0,False,14,0
2,1166795967,20160815,20160816,800,-0.159817,-0.785714,-0.789077,-0.785714,0.633339,0.879566,0.597355,1.417726,0.566608,1.127686,1.283000,0.796019,0.995784,1.413689,0.218041,1.083368,1.149542,1.474626,0.972831,1.088646,1.663667,1.614732,0.0,1.345414,1.258846,1.187768,2.135049,1.535705,1.651898,1.37116,1.694416,1.833908,1.383186,0.975757,0.987423,1.48969,1.549485,1.210805,1.061204,1.279428,1.540031,1.131879,1.213531,1.571267,0.955697,0.0,0.236189,1.564333,1.229216,1.374299,0.444511,0.0,0.0,0.0,0.990211,0.895441,0.0,2.033415,0.0,0.154487,0.387764,1.537237,0.626813,0.0,1.147379,1.564688,1.128007,1.76054,1.1189,1.230449,1.441725,2.127261,1.031093,1.284279,1.891631,1.166048,1.596105,1.59585,0.866867,1.638885,1.3785,1.29153,1.00

In [27]:
all_cat.to_csv("./backup1007/multi_index_time_dependent200108.csv")

In [36]:
encoded_all=np.concatenate([encoded_vital,
encoded_hct,
encoded_electro,
encoded_creatinine,
encoded_crp,
encoded_wbc,
encoded_abga,
encoded_liver],axis=1)

In [38]:
plateletcolumn

Index(['Platelet(diff)', 'Platelet(mean)', 'Platelet(max)', 'Platelet(min)'], dtype='object')

In [39]:
name=outcome[['idx','admday','labday','time(round)','Platelet(diff)', 'Platelet(mean)', 'Platelet(max)', 'Platelet(min)']].to_numpy()

In [40]:
label=outcome.iloc[:,-8:].to_numpy()

In [56]:
encoded_all_=np.concatenate([name,encoded_all,label],axis=1)

encoded_pd=pd.DataFrame(encoded_all_)

In [57]:
encoded_pd.to_csv("./backup1007/scaled_padded_191206.csv")

In [42]:
except_vital = np.concatenate([encoded_hct,encoded_electro,encoded_creatinine,
                               encoded_crp,encoded_wbc,encoded_abga,encoded_liver],axis=1)
except_hct = np.concatenate([encoded_vital,encoded_electro,encoded_creatinine,
                               encoded_crp,encoded_wbc,encoded_abga,encoded_liver],axis=1)
except_electro = np.concatenate([encoded_vital,encoded_hct,encoded_creatinine,
                               encoded_crp,encoded_wbc,encoded_abga,encoded_liver],axis=1)
except_cr = np.concatenate([encoded_vital,encoded_hct,encoded_electro,
                               encoded_crp,encoded_wbc,encoded_abga,encoded_liver],axis=1)
except_crp = np.concatenate([encoded_vital,encoded_hct,encoded_electro,encoded_creatinine,
                               encoded_wbc,encoded_abga,encoded_liver],axis=1)
except_wbc = np.concatenate([encoded_vital,encoded_hct,encoded_electro,encoded_creatinine,
                               encoded_crp,encoded_abga,encoded_liver],axis=1)
except_abga = np.concatenate([encoded_vital,encoded_hct,encoded_electro,encoded_creatinine,
                               encoded_crp,encoded_wbc,encoded_liver],axis=1)
except_crp = np.concatenate([encoded_vital,encoded_hct,encoded_electro,encoded_creatinine,
                               encoded_crp,encoded_wbc,encoded_abga],axis=1)


In [45]:
all_column = {"vital":except_vital,
             "hct":except_hct,
             "electro":except_electro,
             "creatinine":except_cr,
             "crp":except_crp,
             "wbc":except_wbc,
             "abga":except_abga,
             "crp":except_crp}

In [60]:
for a in all_column.keys():
    encoded_all_=np.concatenate([name,all_column[a],label],axis=1)
    encoded_pd=pd.DataFrame(encoded_all_)
    encoded_pd.to_csv("./backup1007/except_{}_200107.csv".format(a))

In [58]:
all_column.keys()

dict_keys(['vital', 'hct', 'electro', 'creatinine', 'crp', 'wbc', 'abga'])